In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#### не помещается
model_name = "Vikhrmodels/Vikhr-Llama-3.2-1B-instruct" 

# Загрузка токенайзера
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Загрузка модели и помещение на GPU
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()


def empty_gpu_cache():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.reset_accumulated_memory_stats()


# Проверка устройства
print(f"Модель размещена на: {next(model.parameters()).device}")

Модель размещена на: cuda:0


In [3]:
#Архитектура модели

print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [4]:
for i, (name, param) in enumerate(model.named_parameters()):
    if i<5:
        print(f"Parameter name: {name}")
        print(f"Shape: {param.shape}")
        print(f"Values: {param[:5]}")
        print()


Parameter name: model.embed_tokens.weight
Shape: torch.Size([128256, 2048])
Values: tensor([[ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334],
        [ 0.0237, -0.0229,  0.0199,  ..., -0.0095, -0.0022, -0.0396],
        [ 0.0145,  0.0106,  0.0098,  ...,  0.0068, -0.0116,  0.0058],
        [-0.0219,  0.0072, -0.0017,  ...,  0.0031,  0.0103, -0.0023],
        [ 0.0234, -0.0337, -0.0315,  ..., -0.0240,  0.0016, -0.0067]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Parameter name: model.layers.0.self_attn.q_proj.weight
Shape: torch.Size([2048, 2048])
Values: tensor([[-0.0183,  0.0056,  0.0255,  ..., -0.0079, -0.0135,  0.0201],
        [ 0.0115,  0.0589,  0.0556,  ..., -0.0330, -0.0160,  0.0111],
        [ 0.0177,  0.0158,  0.0342,  ..., -0.0387, -0.0378, -0.0280],
        [-0.0069, -0.0441, -0.0418,  ...,  0.0210,  0.0214,  0.0312],
        [-0.0294, -0.1022, -0.0605,  ...,  0.0638,  0.0403,  0.0161]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Parameter name: m

In [5]:
#Количество параметров
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 1235814400
Trainable parameters: 1235814400


In [6]:
def get_answer(prompt, tokens_number = 1512, temp = 0.3, top_k = 50, top_p = 0.95):
    
    input_ids = tokenizer.apply_chat_template([{"role": "user", "content": prompt}], truncation=True,
                                          add_generation_prompt=False, padding=False, return_tensors="pt").to("cuda")
    with torch.no_grad():
        
        outputs = model.generate(
                input_ids=input_ids,
                max_new_tokens=tokens_number,
                do_sample=True,
                temperature=temp,
                top_k=top_k,
                top_p=top_p,
                pad_token_id=tokenizer.pad_token_id
        )

    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

    answer = decoded_output.split('<|start_header_id|>assistant<|end_header_id|>')[-1].replace('<|eot_id|>', '').strip()
    
    return answer

    

In [8]:
print('Промпт:')
prompt = 'Сколько будет 2+2*2?'
print(prompt)
print()
print()
print(100*'-')
print('Ответ:')
print()
print(get_answer(prompt, temp = 0.1))

Промпт:
Сколько будет 2+2*2?


----------------------------------------------------------------------------------------------------
Ответ:

Для решения данной математической задачи выполним следующие шаги:

1. Сначала умножим 2 на 2, чтобы получить результат умножения.
2. Затем сложим полученный результат с 2.

Итак, шаг за шагом:

1. \(2 \times 2 = 4\)
2. \(4 + 2 = 6\)

Таким образом, \(2 + 2 \times 2 = 6\).


In [1]:
LLM(prompt, context)

NameError: name 'f' is not defined

In [ ]:
text -> text

### Не знает что такое ГЗ в бауманке. Нужен будет датасет состоящий из сленга и дообучиться на нём.
### Не понимает что дом физики находится внутри гз и отвечает невпопад. 
## Но в целом модель понимает суть вопросов, структура ответа правильная, нужно только дополнить знаниями.